In [1]:
import clickhouse_connect
import pandas as pd
import joblib
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
import numpy as np
import pickle
import gzip
import os
import boto3
from dotenv import load_dotenv
load_dotenv()

In [19]:
# AWS CREDENTIALS
bucket_name = os.getenv('AWS_BUCKET_NAME')
s3_transactions_path = os.getenv('TRANSACTIONS_DF_PATH')
s3_model_path = os.getenv('MODEL_PKL')
aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
aws_session_token=os.getenv("AWS_SESSION_TOKEN")

s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=aws_session_token
)

In [2]:
client = clickhouse_connect.get_client(host=os.getenv('CLICKHOUSE_HOST'), 
                                        user=os.getenv('CLICKHOUSE_USERNAME'), 
                                        password=os.getenv('CLICKHOUSE_PASSWORD'),
                                        port=int(os.getenv('CLICKHOUSE_PORT')),
                                        database=os.getenv('CLICKHOUSE_DATABASE'),
                                        interface='http'
                                        )

In [3]:
def load_data_from_clickhouse(client):
    query = f"""
    SELECT 
      *
    FROM 
        default.vw_recommendation
    """

    result = client.query(query)

    transactions = pd.DataFrame(result.result_rows, columns=result.column_names)
    
    return transactions

In [4]:
transactions = load_data_from_clickhouse(client)

In [5]:
transactions.loc[transactions['categoria'] == 'LÃ¡bios', 'categoria'] = 'Lábios'
transactions.loc[transactions['sub_categoria'] == 'MÃ¡scara', 'sub_categoria'] = 'Máscara'
transactions.loc[transactions['sub_categoria'] == 'Sabonete lÃ­quido', 'sub_categoria'] = 'Sabonete líquido'

In [7]:
basket = pd.pivot_table(data=transactions,index='cod_transacao',columns='sub_categoria',values='quantidade', aggfunc='sum',fill_value=0)
basket_np = basket.to_numpy()
basket_sets_np = np.where(basket_np > 0, 1, 0)
basket_sets = pd.DataFrame(basket_sets_np, index=basket.index, columns=basket.columns)
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)
rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

c:\Users\Inteli\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [10]:
with open(os.getcwd() + '/data/cross_selling_model.pkl', 'wb') as dataframe:
    pickle.dump(rules_mlxtend, dataframe)

In [ ]:
parquet_file_path = os.path.join(os.getcwd(), 'data', 'transactions.parquet')
transactions.to_parquet(parquet_file_path, compression='gzip')

In [ ]:
s3.upload_file(parquet_file_path, bucket_name, s3_transactions_path)

In [21]:
pkl_file_path = os.getcwd() + '/data/cross_selling_model.pkl'
s3.upload_file(pkl_file_path, bucket_name, s3_model_path)